# Create Initial Lattice for BMG GFA Analysis

Creation of a randomly placed lattice for a binary alloy containing particles A and B. Tunable parameters are listed below. The whole lattice is padded with a separation equal to half of that between individual atoms. 

Tunable parameters:

* `A`, `B`: name of particles A and B
* `fA`: fraction of particle A in system
* `MA`, `MB`: molecular masses of particles A and B
* `sep`: separation between atoms in the system
* `N_xyz`: list of lattice particle count

In [6]:
import os
import random
import numpy as np

In [8]:
def get_random_atom_list(N_atoms, fraction):
  
  N_A = round(fraction * N_atoms)
  N_B = round((1 - fraction) * N_atoms)
  A = [1] * N_A
  B = [2] * N_B
  if N_A + N_B != N_atoms:
    if fraction > 0.5:
      A += [1]
    else:
      B += [2]
  atoms = A + B
  random.shuffle(atoms)
  
  return atoms


def create_initial_configuration(A, B, fA, MA, MB, sep, N_xyz):

  N_xyz = np.array(N_xyz)
  N_atoms = np.prod(N_xyz)
  L_xyz = N_xyz * sep
  
  mA = MA / max(MA, MB)
  mB = MB / max(MA, MB)
  
  atoms = get_random_atom_list(N_atoms, fA)
  filename = f'{A}{B}-{fA:.2f}.lammps'
  
  body = f'{filename} - created by Oion; units = lj' + \
  '\n' + f'' + \
  '\n' + f'{N_atoms} atoms' + \
  '\n' + f'0 bonds' + \
  '\n' + f'0 angles' + \
  '\n' + f'0 dihedrals' + \
  '\n' + f'0 impropers' + \
  '\n' + f'' + \
  '\n' + f'2 atom types' + \
  '\n' + f'' + \
  '\n' + f'0.000000 {L_xyz[0]:.6f} xlo xhi' + \
  '\n' + f'0.000000 {L_xyz[1]:.6f} ylo yhi' + \
  '\n' + f'0.000000 {L_xyz[2]:.6f} zlo zhi' + \
  '\n' + f'' + \
  '\n' + f'Masses' + \
  '\n' + f'' + \
  '\n' + f'1	{mA:.8f}	# A' + \
  '\n' + f'2	{mB:.8f}	# B' + \
  '\n' + f'' + \
  '\n' + f'Atoms' + \
  '\n' + f'' + \
  '\n' + f''
  
  i = 0
  for Nx_i in range(N_xyz[0]):
    for Ny_i in range(N_xyz[1]):
      for Nz_i in range(N_xyz[2]):
        pos = (np.array([Nx_i, Ny_i, Nz_i]) + 0.5) * sep
        line = f'{i + 1}	{i + 1}	{atoms[i]}	0.000000	{pos[0]:.6f}	{pos[1]:.6f}	{pos[2]:.6f}'
        body += line + '\n'
        i += 1

  return filename, body
  
  
def save_initial_configuration(A, B, fA, MA, MB, sep, N_xyz, savepath):
  
  filename, body = create_initial_configuration(A, B, fA, MA, MB, sep, N_xyz)
  savepath = os.path.join(savepath, filename)
  with open(savepath, 'w') as file:
    file.write(body)
    
    
def save_initial_configurations(A, B, fA, MA, MB, sep, N_xyz, savepath):
  for fA in fA:
    save_initial_configuration(A, B, fA, MA, MB, sep, N_xyz, savepath)

In [10]:
save_initial_configurations(
  A='Mg', MA=24.305, 
  B='Ni', MB=58.693,
  sep=2, N_xyz = [10, 10, 10], fA=np.arange(0.05, 0.95 + 0.05, 0.05),
  savepath='/Users/admin/tmp'
)